<a href="https://colab.research.google.com/github/jarvisx17/ASR/blob/main/URDU/ASR_URDU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# First i should thanks [Mehrdad Farahani](https://www.kaggle.com/m3hrdadfi) for his awsome [notebook](https://colab.research.google.com/github/m3hrdadfi/notebooks/blob/main/Fine_Tune_XLSR_Wav2Vec2_on_Persian_ShEMO_ASR_with_%F0%9F%A4%97_Transformers_ipynb.ipynb#scrollTo=-ZOtWbQRy0Uk) in google colab.

In [ ]:
# %%capture
# #!conda install --yes pytorch
# #!pip install --upgrade torchaudio
# # !pip install librosa
# # !pip install numpy
# # !pip install pandas
# !pip install transformers
# !pip install datasets
# !pip install hazm
# !pip install num2fawords

# let's importing excel file as a dataframe

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import json

# username = "" #@param {type: "string"}
# api_key = "" #@param {type: "string"}


# if username and api_key:
#     token = {"username": username, "key": api_key}

#     !mkdir ~/.kaggle
#     !mkdir /content/.kaggle
#     with open('/content/.kaggle/kaggle.json', 'w') as f:
#         json.dump(token, f)

#     !cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
#     !chmod 600 /root/.kaggle/kaggle.json

#     print('Your are ready to use kaggle API!')

In [ ]:
import pandas as pd
df = pd.read_csv('/kaggle/input/medical-transcription-and-classification-dataset/Medical_Asr.csv')
# df.to_csv('dataset.csv', index=False)
print(len(df))
df.head()

6661


,Unnamed: 0,file_name,phrase,prompt
0,0,1249120_43453425_58166571.wav,When I remember her I feel down,Emotional pain
1,1,1249120_43719934_43347848.wav,When I carry heavy things I feel like breaking...,Hair falling out
2,2,1249120_43719934_53187202.wav,there is too much pain when i move my arm,Heart hurts
3,3,1249120_31349958_55816195.wav,My son had his lip pierced and it is swollen a...,Infected wound
4,4,1249120_43719934_82524191.wav,My muscles in my lower back are aching,Infected wound


### Now let's clean our excel file

* first I will remove `target` column as I don't need it

In [ ]:
df = df.drop(['Unnamed: 0','prompt'], axis=1)
len(df)

6661

In [ ]:
df['phrase'] = df['phrase'].str.replace('\d+', '')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [ ]:
df['phrase'] = df['phrase'].str.lower()

now I will check if we have any null raws

In [ ]:
df.isna().sum()

file_name    0
phrase       0
dtype: int64

So we don't have any null raw in the excel file

As there might be duplicated items in the excel, I will remove them (Remove duplicated Raws)

In [ ]:
df = df.drop_duplicates(keep=False)
len(df)

6661

# Now let's create our `train` and `test` dataset
### I will use %80 of data as train dataset and %20 of the data as test dataset

In [ ]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# we have to reset index to avoid any future mistakes or any kinds of error
train_df.reset_index(inplace=True)
test_df.reset_index(inplace=True)

train_df = train_df.drop('index', axis=1)
test_df = test_df.drop('index', axis=1)

len(df), len(train_df), len(test_df)

(6661, 5328, 1333)

In [ ]:
train_df.head()

,file_name,phrase
0,1249120_34824899_80106247.wav,i have acne all over my face
1,1249120_43604449_107898973.wav,i have a problem in seeing objects it is too d...
2,1249120_43719934_89749737.wav,i cant sleep because of cough
3,1249120_39142299_32213487.wav,i have a very rash sensation close to my arms
4,1249120_14353703_45949288.wav,i feel a burning sensation in my guts about h...


### Let's check our data. I will call them randomly from datasets

p.s. You can run these two cells bellow and see diffrented result as I used random numbers

In [ ]:
import IPython, random
rand_int = random.randint(0, len(train_df))
file_name = train_df['file_name'][rand_int]
print(train_df['phrase'][rand_int])
IPython.display.Audio('/kaggle/input/medical-transcription-and-classification-dataset/Data/Data/'+file_name)


when i remember her i feel down


In [ ]:
import IPython, random
rand_int = random.randint(0, len(test_df))
file_name = test_df['file_name'][rand_int]
print(test_df['phrase'][rand_int])
IPython.display.Audio('/kaggle/input/medical-transcription-and-classification-dataset/Data/Data/'+file_name)

i have difficulty moving my neck


## Now we need to fix the path of each file because later, we want to use these path to send the files to the model


### Let's check one of the current file's path

In [ ]:
train_df['file_name'][0]

'1249120_34824899_80106247.wav'

### I will define the `fix_path` methode to replace a path file that model can easily find `mp3` files

In [ ]:
def fix_path(dir, text):
    return str(dir+text)

We want to get our files by their address into the hard disk and then save them as csv. So later we can use `load_dataset` method with these csv files

In [ ]:
from datasets import load_dataset
import glob

save_path = '/kaggle/input/medical-transcription-and-classification-dataset/Data/Data/'

for i in range(len(train_df)): # Replacing old path to new path for train dataset

    train_df['file_name'][i] = fix_path(save_path, train_df['file_name'][i])
print(train_df['file_name'][0])

# save new train dataset as csv
train_df.to_csv(f"train.csv", sep="\t", encoding="utf-8", index=False)


for i in range(len(test_df)): # Replacing old path to new path for test dataset

    test_df['file_name'][i] = fix_path(save_path, test_df['file_name'][i])
print(test_df['file_name'][0])

# Save new test dataset as csv
test_df.to_csv(f"test.csv", sep="\t", encoding="utf-8", index=False)

/kaggle/input/medical-transcription-and-classification-dataset/Data/Data/1249120_34824899_80106247.wav
/kaggle/input/medical-transcription-and-classification-dataset/Data/Data/1249120_28006404_28326999.wav


It seems the new path are good

# Now we have to load our train and test datasets

In [ ]:
common_voice_train = load_dataset("csv", data_files={"train": "train.csv"}, delimiter="\t")["train"]
common_voice_test = load_dataset("csv", data_files={"test": "test.csv"}, delimiter="\t")["test"]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-d7b841cd0feaa226/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-dcb98295be91bf0d/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

### Here I did define some method to normalize the text.

but before define them, I need to install `hazm` and `num2fawords` library to use it in the *normalzier*

In [ ]:
import hazm
import re
from num2fawords import words
import string

_normalizer = hazm.Normalizer()

chars_to_ignore = [
    ",", "?", ".", "!", "-", ";", ":", '""', "%", "'", '"', "�",
    "#", "!", "؟", "?", "«", "»", "،", "(", ")", "؛", "'ٔ", "٬",'ٔ', ",", "?",
    ".", "!", "-", ";", ":",'"',"“", "%", "‘", "”", "�", "–", "…", "_", "”", '“', '„',
    'ā', 'š',
]

# In case of farsi
chars_to_ignore = chars_to_ignore + list(string.ascii_lowercase + string.digits)

def multiple_replace(text, chars_to_mapping):
    pattern = "|".join(map(re.escape, chars_to_mapping.keys()))
    return re.sub(pattern, lambda m: chars_to_mapping[m.group()], str(text))

def remove_special_characters(text, chars_to_ignore_regex):
    text = re.sub(chars_to_ignore_regex, '', text).lower() + " "
    return text

def normalizer(batch, chars_to_ignore=chars_to_ignore):
    chars_to_ignore_regex = f"""[{"".join(chars_to_ignore)}]"""
    text = batch["phrase"].lower().strip()
    text = _normalizer.normalize(text)
    text = remove_special_characters(text, chars_to_ignore_regex)
    text = re.sub(" +", " ", text)
    _text = []
    for word in text.split():
        try:
            word = int(word)
            _text.append(words(word))
        except:
            _text.append(word)

    text = " ".join(_text) + " "
    text = text.strip()

    if not len(text) > 0:
        return None

    batch["phrase"] = text

    return batch

### Now I will apply normalizer on our text in the train and test sets

In [ ]:
print(common_voice_train[0]["phrase"])
print(common_voice_test[0]["phrase"])

common_voice_train = common_voice_train.map(normalizer, fn_kwargs={"chars_to_ignore": chars_to_ignore})
common_voice_test = common_voice_test.map(normalizer, fn_kwargs={"chars_to_ignore": chars_to_ignore})

print(common_voice_train[0]["phrase"])
print(common_voice_test[0]["phrase"])


i have acne all over my face
i feel joint pain every time i move


  0%|          | 0/5328 [00:00<?, ?ex/s]

  0%|          | 0/1333 [00:00<?, ?ex/s]

i have acne all over my face
i feel joint pain every time i move


In [ ]:
def extract_all_chars(batch):
    all_text = " ".join(batch["phrase"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, remove_columns=common_voice_test.column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

## Now it's time to create processor files so we can download them after fine-tune and use it

In [ ]:
vocab_list = list(sorted(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0])))
vocab_list = [vocab for vocab in vocab_list if vocab not in [" ", "\u0307"]]
print(len(vocab_list))
print(vocab_list)

37
['\n', '!', "'", ',', '-', '.', ':', '?', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'ï', '√']


In [ ]:
special_vocab = ["<pad>", "<s>", "</s>", "<unk>", "|"]
vocab_dict = {v: k for k, v in enumerate(special_vocab + vocab_list)}
print(len(vocab_dict))
print(vocab_dict)

42
{'<pad>': 0, '<s>': 1, '</s>': 2, '<unk>': 3, '|': 4, '\n': 5, '!': 6, "'": 7, ',': 8, '-': 9, '.': 10, ':': 11, '?': 12, '`': 13, 'a': 14, 'b': 15, 'c': 16, 'd': 17, 'e': 18, 'f': 19, 'g': 20, 'h': 21, 'i': 22, 'j': 23, 'k': 24, 'l': 25, 'm': 26, 'n': 27, 'o': 28, 'p': 29, 'q': 30, 'r': 31, 's': 32, 't': 33, 'u': 34, 'v': 35, 'w': 36, 'x': 37, 'y': 38, 'z': 39, 'ï': 40, '√': 41}


In [ ]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [ ]:
# m3hrdadfi/wav2vec2-large-xlsr-persian-v2
model_name_or_path = "facebook/wav2vec2-base"
save_dir = './new_model/'

In [ ]:
import os

In [ ]:
from transformers.trainer_utils import get_last_checkpoint

In [ ]:
last_checkpoint = None
if os.path.exists(save_dir):
    last_checkpoint = get_last_checkpoint(save_dir)

print(last_checkpoint if last_checkpoint else str(None))

None


In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("/kaggle/working/vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [ ]:
from transformers import Wav2Vec2CTCTokenizer

if not os.path.exists(save_dir) and not model_name_or_path:
    print("Load from scratch")
    tokenizer = Wav2Vec2CTCTokenizer(
        "./vocab.json",
        bos_token="<s>",
        eos_token="</s>",
        unk_token="<unk>",
        pad_token="<pad>",
        word_delimiter_token="|",
        do_lower_case=False
    )
else:
    print(f"Load from {model_name_or_path}")
    tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(model_name_or_path)

Load from facebook/wav2vec2-base


In [ ]:
text = "i'm not feeling well."
print(" ".join(tokenizer.tokenize(text)))
print(tokenizer.decode(tokenizer.encode(text)))

i ' m | n o t | f e e l i n g | w e l l .
i'm not feling wel.


In [ ]:
from transformers import Wav2Vec2FeatureExtractor

if not os.path.exists(save_dir) and not model_name_or_path:
    print("Load from scratch")
    feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)
else:
    print(f"Load from {model_name_or_path}")
    feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_name_or_path)

Load from facebook/wav2vec2-base


In [ ]:
from transformers import Wav2Vec2Processor

if not os.path.exists(save_dir) and not model_name_or_path:
    print("Load from scratch")
    processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)
else:
    print(f"Load from {model_name_or_path}")
    processor = Wav2Vec2Processor.from_pretrained(model_name_or_path)

Load from facebook/wav2vec2-base


In [ ]:
if len(processor.tokenizer.get_vocab()) == len(processor.tokenizer):
    print(len(processor.tokenizer))

32


In [ ]:
if not os.path.exists('./new_model/processor/'):
    print("Saving ...")
    processor.save_pretrained('./new_model/processor/')
    print("Saved!")

## Next step is to turn our audio files to array so that we can start processing on them

In [ ]:
import torchaudio
import librosa
import numpy as np

target_sampling_rate = 16_000

def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["file_name"])
    speech_array = speech_array.squeeze().numpy()
    speech_array = librosa.resample(np.asarray(speech_array), sampling_rate, target_sampling_rate)

    batch["speech"] = speech_array
    batch["sampling_rate"] = target_sampling_rate
    batch["duration_in_seconds"] = len(batch["speech"]) / target_sampling_rate
    batch["target_text"] = batch["phrase"]
    return batch

# You can change the num_proc below, depends on your cpu and cpu cores
common_voice_train = common_voice_train.map(speech_file_to_array_fn, batch_size=4, remove_columns=common_voice_train.column_names, num_proc=4)
common_voice_test = common_voice_test.map(speech_file_to_array_fn, batch_size= 4, remove_columns=common_voice_test.column_names, num_proc=4)

common_voice_train[0].keys()

print(common_voice_train[0]["sampling_rate"])
print(common_voice_test[0]["sampling_rate"])


#0:   0%|          | 0/1332 [00:00<?, ?ex/s]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: Pass orig_sr=44100, target_sr=16000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  # Remove the CWD from sys.path while we load stuff.


#1:   0%|          | 0/1332 [00:00<?, ?ex/s]

#2:   0%|          | 0/1332 [00:00<?, ?ex/s]

#3:   0%|          | 0/1332 [00:00<?, ?ex/s]

...

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: Pass orig_sr=44100, target_sr=16000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  # Remove the CWD from sys.path while we load stuff.


#1:   0%|          | 0/333 [00:00<?, ?ex/s]

#3:   0%|          | 0/333 [00:00<?, ?ex/s]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: Pass orig_sr=48000, target_sr=16000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  # Remove the CWD from sys.path while we load stuff.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: Pass orig_sr=44100, target_sr=16000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  # Remove the CWD from sys.path while we load stuff.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: Pass orig_sr=44100, target_sr=16000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  # Remove the CWD from sys.path while we load stuff.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: Pass orig_sr=48000, target_sr=16000 as keyword args. From version 0.10 passing these as positional arguments will resul

16000
16000


### Make ready our dataset to send them to the model

In [ ]:
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor, Wav2Vec2Processor, Wav2Vec2ForCTC, Wav2Vec2Processor

model_name_or_path = 'm3hrdadfi/wav2vec2-large-xlsr-persian-shemo'
processor = Wav2Vec2Processor.from_pretrained(model_name_or_path)

def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values

    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

_common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names, batch_size=4, num_proc=4, batched=True)
_common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names, batch_size=4, num_proc=4, batched=True)


Downloading:   0%|          | 0.00/158 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/404 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

/opt/conda/lib/python3.7/site-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


Downloading:   0%|          | 0.00/398 [00:00<?, ?B/s]

...

#1:   0%|          | 0/333 [00:00<?, ?ba/s]

#3:   0%|          | 0/333 [00:00<?, ?ba/s]

#2:   0%|          | 0/333 [00:00<?, ?ba/s]

/opt/conda/lib/python3.7/site-packages/transformers/feature_extraction_utils.py:168: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/opt/conda/lib/python3.7/site-packages/transformers/feature_extraction_utils.py:168: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)


ValueError: Unable to create tensor, you should probably activate padding with 'padding=True' to have batched tensors with the same length.

### Now I will define the parameters of model, processor, metric and the trainer object to modify train

In [ ]:
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
import torch

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)


In [ ]:
!pip install jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 4.1 MB/s eta 0:00:0000:0100:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 19.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.7/113.7 kB 10.3 MB/s eta 0:00:00


In [ ]:
from datasets import load_metric
import os

wer_metric = load_metric("wer")
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    if isinstance(label_str, list):
        if isinstance(pred_str, list) and len(pred_str) == len(label_str):
            for index in random.sample(range(len(label_str)), 3):
                print(f'reference: "{label_str[index]}"')
                print(f'predicted: "{pred_str[index]}"')

        else:
            for index in random.sample(range(len(label_str)), 3):
                print(f'reference: "{label_str[index]}"')
                print(f'predicted: "{pred_str}"')

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

os.environ["WANDB_DISABLED"] = "true"


In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    # "facebook/wav2vec2-large-xlsr-53" if not last_checkpoint else last_checkpoint,
    model_name_or_path,
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    gradient_checkpointing=True,
    ctc_loss_reduction="mean",
    ctc_zero_infinity=True,
    bos_token_id=processor.tokenizer.bos_token_id,
    eos_token_id=processor.tokenizer.eos_token_id,
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer.get_vocab())

)

# model.freeze_feature_extractor()
save_dir = './new_model/'
from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir=save_dir,
    group_by_length=True,

    per_device_train_batch_size=10,
    per_device_eval_batch_size=10,
    seed=0,
    # because we are using little batch-size, we set accumulation step to 2 so every 2 epoch the gradiant will be optimize
    gradient_accumulation_steps=2,
    evaluation_strategy="steps", # Comment this line if you're training with cpu
    num_train_epochs=4,
    fp16=True, # False for using CPU and True for using GPU for training
    # save_steps=10,
    eval_steps=50, # Comment this line if you're training with cpu
    # logging_steps=10,
    learning_rate=1e-4,
    warmup_steps=500,
    # save_total_limit=2,
)

from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=_common_voice_train,
    eval_dataset=_common_voice_test,
    tokenizer=processor.feature_extractor,
)


Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using cuda_amp half precision backend


In [ ]:
trainer.train()
results = {}
metrics = trainer.evaluate()
max_val_samples = len(_common_voice_test)
metrics["eval_samples"] = min(max_val_samples, len(_common_voice_test))

trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

trainer.save_model()

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3995
  Num Epochs = 4
  Instantaneous batch size per device = 10
  Total train batch size (w. parallel, distributed & accumulation) = 20
  Gradient Accumulation steps = 2
  Total optimization steps = 800


Step,Training Loss,Validation Loss,Wer
50,No log,0.921870,0.413830
100,No log,0.444083,0.380738
150,No log,0.364200,0.333887
200,No log,0.315803,0.308386
250,No log,0.297430,0.287510
300,No log,0.283164,0.270549
350,No log,0.263948,0.250385
400,No log,0.254678,0.239948
450,No log,0.248137,0.230340
500,0.554400,0.239291,0.215396


***** Running Evaluation *****
  Num examples = 999
  Batch size = 10


reference: "امکان دارد این بسته را وزن کنید لطفا"
predicted: "امکان داد این بدسرا وض کنید لطفا"
reference: "ریچل چرا تماس نگرفت"
predicted: "ریچل چرا تماس نگرفت"
reference: "سعی کردم با او تماس بگیرم اما نتوانستم وصل بشوم"
predicted: "سعی کردم با او تماس بگیرما نتبنستمو وصل بشم"


***** Running Evaluation *****
  Num examples = 999
  Batch size = 10


...

reference: "ما آماده حرکت هستیم سفر بیخطر"
predicted: "ما آماده حرکت هستیم سفر بی خطر"
reference: "من باید با سفارتخانه تماس بگیرم"
predicted: "من باید با سفارتخانه تماس بگیرم"
reference: "بعد با گرد هم آوردن گروههای مختلف فعالشان کنیم"
predicted: "بعد با گرد هم آوردن گروههای مختلف فعالشان کنیم"


***** Running Evaluation *****
  Num examples = 999
  Batch size = 10


reference: "کجا می توانم تماس بگیرم"
predicted: "کجا می توانم تماس بگیرم"
reference: "این بسته را با پست هوایی بفرستید"
predicted: "این بسته را با پست هوایی بفرستیدتتت"
reference: "اومدم که گوشی رو بگیرم"
predicted: "اومدم که گوشی رو بگیرم"


***** Running Evaluation *****
  Num examples = 999
  Batch size = 10


reference: "تو هند اینجوری پرونده اش بسته شد"
predicted: "تو هند اینجوری فرانده اشبسته شد"
reference: "چیزهایی که میتواند سبب خطا در حافظه شود را تفکیک کردهام"
predicted: "چیزهایی که میتواند سبب خطا در حافظه شود را تفکیک کرده اند"
reference: "خطاب به موش می بینی چه زندگی برامون درست کردی"
predicted: "خطاب به موش می بینی چه زندگی برامون درست کردی"
